In [1]:
from DatasetHandler.BiwiBrowser import *
from LSTM_VGG16.LSTM_VGG16Helper import *
%matplotlib inline

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
output_begin = 4
num_outputs = 1

In [3]:
timesteps = 26 # TimeseriesGenerator Handles overlapping
in_epochs = 1
out_epochs = 10
train_batch_size = 1
test_batch_size = 1

In [4]:
subjectList = [1, 2, 3, 4, 5, 7, 8, 11, 12, 14] # except [6, 13, 10, ]
testSubjects = [9]

In [5]:
num_datasets = len(subjectList)

In [6]:
def getImageModel(num_outputs = num_outputs):
    inp = BIWI_Frame_Shape
    vgg_model = VGG16(weights='imagenet', include_top=False, input_shape = BIWI_Frame_Shape)
    rnn = Sequential()
    rnn.add(TimeDistributed(vgg_model, input_shape=(timesteps, inp[0], inp[1], inp[2])))
    rnn.add(TimeDistributed(Flatten()))
    
    rnn.add(TimeDistributed(Dense(1024, activation='relu')))
    rnn.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2, name='img_lstm'))

    for layer in rnn.layers[:15]:
        layer.trainable = False
   # rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return rnn

In [7]:
def getAngleModel(num_outputs = num_outputs):
    rnn = Sequential()
    rnn.add(LSTM(26, dropout=0.2, recurrent_dropout=0.2, input_shape=(timesteps, num_outputs), name='ang_lstm'))
  #  rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return rnn

In [8]:
def getFinalModel(num_outputs = num_outputs):
    imageModel = getImageModel(num_outputs = num_outputs)
    angleModel = getAngleModel(num_outputs = num_outputs)
    merged = concatenate([imageModel.output, angleModel.output])#TimeDistributed()C, axis=-1, mode='concat'))  keras.layers.merge
    dense = Dense(num_outputs)(merged)
    rnn = Model([imageModel.input, angleModel.input], dense)
   # rnn.add(merged)
    rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return rnn

In [9]:
keras.backend.clear_session()
full_model = getFinalModel(num_outputs = num_outputs)

In [10]:
#full_model.summary()

In [11]:
#[1]
test_generators, test_labelSets = getTestBiwiForFinalModel(testSubjects, timesteps, False, output_begin, num_outputs, batch_size = test_batch_size)
test_gen, test_labels = test_generators[0], test_labelSets[0] 

All frames and annotations from 1 datasets have been read by 2019-01-02 04:41:23.714118


In [ ]:
full_model = trainFinalModelForEpochs(full_model, out_epochs, subjectList, testSubjects, timesteps, False, output_begin, num_outputs, batch_size = train_batch_size, in_epochs = in_epochs)

All frames and annotations from 10 datasets have been read by 2019-01-02 04:41:38.854337
1. set (Dataset 12) being trained for epoch 1!
Epoch 1/1
732/732 [==============================] - 547s 747ms/step - loss: 0.1409 - mean_absolute_error: 0.2971
2. set (Dataset 3) being trained for epoch 1!
Epoch 1/1
401/730 [===============>..............] - ETA: 3:59 - loss: 0.0804 - mean_absolute_error: 0.2257

In [ ]:
predictions = full_model.predict_generator(test_gen, steps= len(test_labels[timesteps:, :1]), verbose=2)
output1 = numpy.concatenate((test_labels[timesteps:, :1], predictions[:, :1]), axis=1)

In [ ]:
def slide(m, x):
        m[0, :-1] = m[0, 1:]
        m[0, -1] = x
        return m

In [ ]:
cur_pred = np.zeros((1, 26, 1))
pred = []
c =0
for (inputMatrix, inputLabels), outputLabels in test_gen:
    im = inputMatrix.reshape(inputMatrix.shape[:1] + (1,) + inputMatrix.shape[1:])
    #l = inputLabels.reshape(inputLabels.shape[:1] + (1,) + inputLabels.shape[1:])
    c+=1
    if c > len(test_labels): break
    for i in range(len(inputLabels)):#50
        p = full_model.predict([im[i], cur_pred])
        pred.append(p)
        cur_pred = slide(cur_pred, p)
        

In [ ]:
pr = np.array(pred)[:, :1]

In [ ]:
pr.shape

In [ ]:
output1 = numpy.concatenate((test_labels[:len(pred), :1], np.array(pred)[:, 0, :1]), axis=1)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(output1)